In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [3]:
#statistics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16

#data structures
import numpy as np
import PIL.Image as Image
from tensorflow.keras.preprocessing import image

#visualization
import matplotlib.pyplot as plt

#other
import os

print(tf.__version__) #1.14
print(keras.__version__) #2.2.4

1.14.0
2.2.4-tf


In [4]:
tf.enable_eager_execution()